<a href="https://colab.research.google.com/github/awanshrestha/fake-news-classification/blob/main/fake_news_classification_naive_bayes_awan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Naive Bayes**

#### **Importing All Necessary Library**

In [4]:
import pandas as pd
import numpy as np
!pip install stop_words
import re, nltk, stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix, classification_report

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=20f97647f5d9e3356919778b82bf71e3fa3ed63ec6014124ee5c0bcf114a7837
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### **Gathering Data**

In [6]:
fake = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fake.csv')
real = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/True.csv')
fake['fake?'] = 1
real['fake?'] = 0
df = pd.concat([fake,real])
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,title,text,subject,date,fake?
0,"After Nevada, Sanders faces struggle to broade...",WASHINGTON (Reuters) - Bernie Sanders’ high-fl...,politicsNews,"February 21, 2016",0
1,Putin and Trump to Potentially Meet in Slovenia,"21st Century Wire says The Russian President, ...",US_News,"February 10, 2017",1
2,"Behind Donald Trump, a son-in-law who is also ...",NEW YORK (Reuters) - Before introducing Donald...,politicsNews,"April 4, 2016",0
3,Saudi Arabia suspends any dialogue with Qatar:...,DUBAI (Reuters) - Saudi Arabia on Saturday sus...,worldnews,"September 8, 2017",0
4,Guatemala prosecutors target ex-president for ...,GUATEMALA CITY (Reuters) - Guatemala s attorne...,worldnews,"October 6, 2017",0


#### **Data Cleaning : Removing Stop Words, Unnecessary Characters and Stemming**

In [7]:
def transformation(text):
    text = re.sub('[^A-Za-z]',' ',text)
    return ' '.join([nltk.stem.PorterStemmer().stem(word) for word in text.lower().split() if word not in stop_words.get_stop_words('english')])
text = df['text'].apply(transformation)
labels = df['fake?']

#### **Encoding words to numbers using Bag of Words**

In [8]:
vectorize = CountVectorizer(max_features=200,ngram_range=(1,3))
x = vectorize.fit_transform(text).toarray()
y = np.array(labels)

#### **Spliting dataset for Training and Testing**

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2,random_state=34)
print('\nTotal number of samples in the Train Dataset :',xtrain.shape[0])
print('Total number of samples in the Test Dataset :',xtest.shape[0])


Total number of samples in the Train Dataset : 35918
Total number of samples in the Test Dataset : 8980


#### **Training Bernoulli Naive Bayes Classification Model**

In [10]:
bnb = BernoulliNB().fit(xtrain,ytrain)
print('\nAccuracy score :',bnb.score(xtest,ytest))


Accuracy score : 0.9804008908685968


#### **Performance Analysis of Model**

In [11]:
ypred = bnb.predict(xtest)
print('Confusion Metrics : \n\n',confusion_matrix(ytest,ypred),'\n\n')
print('Classification Report :\n\n',classification_report(ytest,ypred))

Confusion Metrics : 

 [[4309   74]
 [ 102 4495]] 


Classification Report :

               precision    recall  f1-score   support

           0       0.98      0.98      0.98      4383
           1       0.98      0.98      0.98      4597

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



#### **Defining the function that checks the News and return weather it is Fake or Real**

In [12]:
def predict(text):
    text = transformation(text)
    text = vectorize.transform([text])
    if bnb.predict(text)[0] == 1:
        print('This is fake news.')
    else:
        print('This is real news.')

#### **Testing the Function and Classification of News - Real or Fake?**

In [13]:
corpus1 = '''One of Donald Trump s favorite punching bags is CNN. He even once tweeted a GIF image of himself punching a person with a CNN logo superimposed over the head 
            indicating that he d like to enact violence against CNN s reporters. Then there was the time he tweeted the  Trump Train roaring over  CNN.  Now, he s back at it 
            this time suggesting that  fake  CNN should be the ones representing America to the world, and that they are doing a bad job. Here is that tweet:.@FoxNews 
            is MUCH more important in the United States than CNN, but outside of the U.S., CNN International is still a major source of (Fake) news, and they represent
            our Nation to the WORLD very poorly. The outside world does not see the truth from them!  Donald J. Trump (@realDonaldTrump) November 25, 2017Of course, 
            it is beneath the dignity of most people to respond to a moronic buffoon like Trump under normal circumstances. However, he is currently squatting in the 
            White House, and has his tiny orange hands on the levers of power   not to mention the nuclear codes   so they have to stoop to a Trumpian level when 
            personally attacked. However, being, well, you know, FIT to be doing the job they are doing, the good folks at CNN Communications fired back at Trump, and 
            their response is nothing short of perfect:It's not CNN's job to represent the U.S to the world. That's yours. Our job is to report the news. #FactsFirst   
            CNN Communications (@CNNPR) November 25, 2017BOOM! Couldn t have asked for a sicker burn than this. And they are right of course   especially the part about
            #FactsFirst. Trump has a problem with the truth, as we all well know. That s what makes what the CNN Communications people replied so fabulous. 
            It is the ultimate truth   something the likes of the pathological orange liar that is Donald Trump knows nothing about.Featured image via  
            Andrew Burton/Getty Images '''



corpus2 = '''WASHINGTON (Reuters) - The No. 2 Republican in the U.S. House of Representatives said on Tuesday the House would likely need to vote again on tax
            legislation on Wednesday morning given that Democrats in the Senate were likely to prevail on a procedural objection. House Majority Leader Kevin McCarthy 
            advised House lawmakers that Senate Democrats were likely to object that the legislation fails to comply with the so-called Byrd rule and were likely to be
            upheld, necessitating a second House vote. “As such, members are further advised that an additional procedural vote on the Motion to Concur is expected 
                tomorrow morning, which will clear the bill for President Trump’s signature,” McCarthy said in a notice to House lawmakers. '''

predict(corpus1)
predict(corpus2)

This is fake news.
This is real news.
